<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2024Project/blob/grupowania-kowalski/xgBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup środowiska


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2024Project.git
#%cd BigData2024Project
#!git checkout your-branch
#%cd ..

Cloning into 'BigData2024Project'...
remote: Enumerating objects: 1877, done.
remote: Counting objects: 100% (579/579), done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 1877 (delta 442), reused 286 (delta 264), pack-reused 1298
Receiving objects: 100% (1877/1877), 69.33 MiB | 12.49 MiB/s, done.
Resolving deltas: 100% (1054/1054), done.
Updating files: 100% (149/149), done.


In [3]:
!chmod 755 /content/BigData2024Project/src/setup.sh
!/content/BigData2024Project/src/setup.sh
!pip install datashader
!pip install holoviews hvplot colorcet
!pip install geoviews

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.1.0
    Uninstalling platformdirs-4.1.0:
      Successfully uninstalled platformdirs-4.1.0
  Attempting uninstall: keyring
    Found existing installation: keyring 23.5.0
    Uninstallin

In [4]:
import sys
sys.path.append('/content/BigData2024Project/src')

In [5]:
from start_spark import initialize_spark
initialize_spark()

# Wczytanie danych

In [6]:
import pandas as pd
from pyspark.sql import SparkSession

from big_mess.loaders import default_loader, load_single_month, load_anotated, save_to_csv, preprocessed_loader

In [7]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [8]:
NASA_an = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_an.csv',sep=';')
NASA_an["pustynia_i_step"] = NASA_an["pustynia"] + NASA_an["step"]

In [10]:
nasa_202305 = load_single_month(spark, year=2023, month=5).toPandas()
nasa_202306 = load_single_month(spark, year=2023, month=6).toPandas()
nasa_202307 = load_single_month(spark, year=2023, month=7).toPandas()
nasa_202308 = load_single_month(spark, year=2023, month=8).toPandas()
nasa_202309 = load_single_month(spark, year=2023, month=9).toPandas()

# Eksperymenty

In [118]:
NASA_an

,lon,lat,pustynia,step,pustynia_i_step
0,-98.4375,49.0625,0,0,0
1,-84.4375,51.9375,0,0,0
2,-100.5625,47.0625,0,0,0
3,-104.6875,48.8125,0,0,0
4,-95.1875,36.5625,0,0,0
...,...,...,...,...,...
1190,-108.0625,42.8125,0,0,0
1191,-103.1875,51.5625,0,0,0
1192,-88.8125,51.5625,0,0,0
1193,-120.5625,51.3125,0,0,0


In [173]:
from sklearn.model_selection import train_test_split

target = 'pustynia_i_step'
X = NASA_an[['lon', 'lat']]
y = NASA_an[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.16, stratify=y, random_state=42)


In [174]:
import pandas as pd
data = nasa_202307
data_columns = [column for column in data.columns if column not in ('lon', 'lat', 'Year', 'Month')]
X_train = X_train.reset_index().merge(data.drop(columns = ['Year', 'Month']), on=['lon', 'lat'], how='inner').set_index('index')
X_test = X_test.reset_index().merge(data.drop(columns = ['Year', 'Month']), on=['lon', 'lat'], how='inner').set_index('index')
y_train = y_train[X_train.index]
y_test = y_test[X_test.index]

In [178]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
X_train[data_columns] = scaler.fit_transform(X_train[data_columns])
X_test[data_columns] = scaler.transform(X_test[data_columns])

In [179]:
from sklearn.preprocessing import LabelEncoder

target_encoder = LabelEncoder()
y_train = pd.Series(target_encoder.fit_transform(y_train), index=y_train.index)
y_test = pd.Series(target_encoder.transform(y_test), index=y_test.index)

In [184]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier(
    objective='binary:logistic',  # For binary classification
    eval_metric='logloss',         # Evaluation metric
    random_state=42                 # Set a random seed for reproducibility
)

xgb_classifier.fit(X_train[data_columns], y_train)
y_train_pred = pd.Series(target_encoder.inverse_transform(xgb_classifier.predict(X_train[data_columns])), index=X_train.index)
y_test_pred = pd.Series(target_encoder.inverse_transform(xgb_classifier.predict(X_test[data_columns])), index=X_test.index)

In [185]:
y_train_pred

index
353    0
167    0
738    0
692    0
517    0
      ..
50     1
704    0
627    0
314    1
731    0
Length: 1002, dtype: int64

In [186]:
from sklearn.metrics import classification_report

print('Training:')
print(classification_report(y_train_pred, y_train))
print('Testing:')
print(classification_report(y_test_pred, y_test))

Training:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       720
           1       1.00      1.00      1.00       282

    accuracy                           1.00      1002
   macro avg       1.00      1.00      1.00      1002
weighted avg       1.00      1.00      1.00      1002

Testing:
              precision    recall  f1-score   support

           0       0.93      0.90      0.91       144
           1       0.72      0.81      0.76        48

    accuracy                           0.88       192
   macro avg       0.83      0.85      0.84       192
weighted avg       0.88      0.88      0.88       192



In [187]:
144/(192)

0.75

# Build a pipeline

In [202]:
from sklearn.base import BaseEstimator, ClassifierMixin
from typing import Literal, Optional
import pandas as pd
class CustomModel(BaseEstimator, ClassifierMixin):

    def __init__(self, data: pd.DataFrame, scaler: Literal['standard','minmax'] = 'standard', random_state:Optional[int] = None):
      self.data = data
      self.data_columns = [column for column in data.columns if column not in ('lon', 'lat', 'Year', 'Month')]
      self.scaler = StandardScaler() if scaler=='standard' else MinMaxScaler()
      self.target_encoder = LabelEncoder()
      self.model = xgb_classifier = XGBClassifier(
          objective='binary:logistic',  # For binary classification
          eval_metric='logloss',        # Evaluation metric
          random_state=random_state     # Set a random seed for reproducibility
      )

    def fit(self, X: pd.DataFrame, y: pd.Series):
      _X = X
      _X = _X.reset_index().merge(data.drop(columns = ['Year', 'Month']), on=['lon', 'lat'], how='inner').set_index('index')
      _X[self.data_columns] = self.scaler.fit_transform(_X[self.data_columns])
      _y = pd.Series(self.target_encoder.fit_transform(y), index=y.index)
      self.model.fit(_X[data_columns], _y)
      return self

    def predict(self, X: pd.DataFrame) -> pd.Series:
      _X = X
      _X = _X.reset_index().merge(data.drop(columns = ['Year', 'Month']), on=['lon', 'lat'], how='inner').set_index('index')
      _X = self.scaler.transform(_X)
      pred_y = self.model.predict(_X)
      pred_y = pd.Series(target_encoder.inverse_transform(pred_y), index=_X.index)
      return pred_y


In [203]:
def check_matching_pairs(df, data):
    # Check if all rows in df have matching pairs in data
    matching_rows = df.merge(data, on=['lon', 'lat'], how='left', indicator=True)
    non_matching_rows = matching_rows[matching_rows['_merge'] == 'left_only']

    if non_matching_rows.empty:
        print("All rows in df have matching pairs in data.")
    else:
        print("Rows in df without matching pairs in data:")
        print(non_matching_rows.drop(columns=['_merge']))

check_matching_pairs(NASA_an, nasa_202308)

All rows in df have matching pairs in data.


In [199]:
NASA_an.drop(965, inplace=True)

KeyError: '[965] not found in axis'

In [204]:
target = 'pustynia_i_step'
X = NASA_an[['lon', 'lat']]
y = NASA_an[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.16, stratify=y, random_state=42)

In [205]:
for data in (nasa_202305, nasa_202306, nasa_202307, nasa_202308, nasa_202309):
  model = CustomModel(data=data)
  model.fit(X_train, y_train)
  y_test_pred = model.predict(X_test)
  print(classification_report(y_test_pred, y_test))

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- lat
- lon
